In [50]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, TimeSeriesSplit
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
from sklearn.preprocessing import  OneHotEncoder,StandardScaler
import pandas_profiling
import datetime
%pylab inline
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv('data/uplift_train.csv')
df1 = pd.read_csv('data/clients.csv')
df2 = pd.read_csv('data/products.csv')
test = pd.read_csv('data/uplift_test.csv')

In [3]:
data_train = pd.merge(df1,df)
miss = data_train[data_train['first_redeem_date'].isna()]
miss['first_redeem_date']=miss['first_issue_date']
data_train.first_redeem_date[miss.index]=miss.first_redeem_date.values
data_train

,client_id,first_issue_date,first_redeem_date,age,gender,treatment_flg,target
0,000012768d,2017-08-05 15:40:48,2018-01-04 19:30:07,45,U,0,1
1,000036f903,2017-04-10 13:54:23,2017-04-23 12:37:56,72,F,1,1
2,00010925a5,2018-07-24 16:21:29,2018-09-14 16:12:49,83,U,1,1
3,0001f552b0,2017-06-30 19:20:38,2018-08-28 12:59:45,33,F,1,1
4,00020e7b18,2017-11-27 11:41:45,2018-01-10 17:50:05,73,U,1,1
...,...,...,...,...,...,...,...
200034,fffe0abb97,2017-11-27 08:56:54,2018-02-11 09:26:08,35,F,0,0
200035,fffe0ed719,2017-09-15 08:53:24,2017-12-12 14:50:12,69,U,0,1
200036,fffea1204c,2018-01-31 16:59:37,2018-03-12 17:02:27,73,F,0,1
200037,fffeca6d22,2017-12-28 11:56:13,2017-12-28 11:56:13,77,F,1,0


In [4]:
data_test=pd.merge(df1,test)
miss = data_test[data_test['first_redeem_date'].isna()]
miss['first_redeem_date']=miss['first_issue_date']
data_test.first_redeem_date[miss.index]=miss.first_redeem_date.values
data_test

,client_id,first_issue_date,first_redeem_date,age,gender
0,000048b7a6,2018-12-15 13:33:11,2018-12-15 13:33:11,68,F
1,000073194a,2017-05-23 12:56:14,2017-11-24 11:18:01,60,F
2,00007c7133,2017-05-22 16:17:08,2018-12-31 17:17:33,67,U
3,00007f9014,2017-08-22 13:35:01,2019-02-15 19:19:18,45,F
4,0000a90cf7,2017-05-20 12:34:50,2017-07-08 12:48:22,45,U
...,...,...,...,...,...
200118,fffeacbe3e,2017-07-19 08:38:17,2017-10-13 10:59:14,47,F
200119,fffece623e,2018-05-13 15:09:30,2018-12-27 19:06:39,67,U
200120,ffff3dfff8,2018-11-01 16:06:52,2018-12-07 17:33:47,56,F
200121,ffffaab9da,2017-08-25 18:43:24,2017-12-08 17:53:19,23,F


In [5]:
data_train['first_issue_date']=pd.to_datetime(data_train['first_issue_date'])
data_train['first_redeem_date']=pd.to_datetime(data_train['first_redeem_date'])
data_test['first_issue_date']=pd.to_datetime(data_test['first_issue_date'])
data_test['first_redeem_date']=pd.to_datetime(data_test['first_redeem_date'])

In [29]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200039 entries, 0 to 200038
Data columns (total 8 columns):
client_id            200039 non-null object
first_issue_date     200039 non-null float64
first_redeem_date    200039 non-null float64
age                  200039 non-null int64
gender               200039 non-null object
treatment_flg        200039 non-null int64
target               200039 non-null int64
fake_age             200039 non-null int64
dtypes: float64(2), int64(4), object(2)
memory usage: 18.7+ MB


In [7]:
data_train['fake_age'] = data_train['age'].apply(lambda x:int((x>90)or(x<16)))
data_test['fake_age'] = data_test['age'].apply(lambda x:int((x>90)or(x<16)))

In [8]:
data_train['first_issue_date'] = data_train['first_issue_date'].apply(lambda x: x.timestamp())
data_train['first_redeem_date'] = data_train['first_redeem_date'].apply(lambda x: x.timestamp())
data_test['first_issue_date'] = data_test['first_issue_date'].apply(lambda x: x.timestamp())
data_test['first_redeem_date'] = data_test['first_redeem_date'].apply(lambda x: x.timestamp())

In [9]:
purch = pd.read_csv('data/purchases.csv',parse_dates=['transaction_datetime'])

In [10]:
user_id = purch.drop_duplicates('client_id').client_id.values

In [11]:
transactions = purch.drop_duplicates('transaction_id')

In [12]:
transactions['transaction_datetime'] = transactions['transaction_datetime'].apply(lambda x: x.timestamp())

In [13]:
#число покупок
a = transactions[['client_id','transaction_id']].values
pok=[]
user=''
count=0
for row in a:
    if row[0]!=user:
        pok.append(count)
        user =row[0]
        count=1
    else:
        count+=1
pokupki = pok[1:]
pokupki.append(count)

In [14]:
#даты последних покупок
dates = purch.drop_duplicates('client_id',keep='last').transaction_datetime.values

In [15]:
#число магазинов
b = transactions[['client_id','store_id']].values
store=[]
user=''
count=0
for row in b:
    if row[0]!=user:
        store.append(count)
        user =row[0]
        count=1
        last_store = row[1]
    else:
        if row[1]!=last_store:
            count+=1
            last_store = row[1]
stores = store[1:]
stores.append(count)

In [16]:
#cредняя сумма покупки
c = transactions[['client_id','purchase_sum']].values
avsum=[]
user=''
count=1
sred=0
for row in c:
    if row[0]!=user:
        avsum.append(sred/count)
        user =row[0]
        count=1
        sred=row[1]
    else:
        count+=1
        sred+=row[1]
average = avsum[1:]
average.append(sred/count)

In [17]:
produkti = purch.product_id.values
useri = purch.client_id.values

In [18]:
alcohol = set(df2[df2['is_alcohol']==1].product_id.values)
owner = set(df2[df2['is_own_trademark']==1].product_id.values)

In [19]:
alc = lambda x: int(x in alcohol)
is_alc = [alc(i) for i in produkti]
own = lambda x: int(x in owner)
is_own = [own(i) for i in produkti]

In [20]:
purch['product_is_alco']=is_alc
purch['product_is_own']=is_own

In [21]:
#число купленного алкоголя и собственной торговой марки
a = transactions[['client_id','transaction_id']].values
alc_count=[]
own_count=[]
user=''
count_alc=0
count_own=0
for i in range(len(useri)):
    if useri[i]!=user:
        alc_count.append(count_alc)
        own_count.append(count_own)
        user =useri[i]
        count_alc=is_alc[i]
        count_own=is_own[i]
    else:
        count_alc+=is_alc[i]
        count_own+=is_own[i]
alc_count = alc_count[1:]
own_count = own_count[1:]
alc_count.append(count_alc)
own_count.append(count_own)

In [25]:
new_features = pd.DataFrame({'client_id':user_id,'transaction_count':pokupki,'av_sum':average,
                             'last_transaction':dates,'store_count':stores,
                             'alc_count':alc_count,'own_count':own_count})

In [30]:
new_features['last_transaction'] = new_features['last_transaction'].apply(lambda x: x.timestamp())

In [33]:
data_train_full = pd.merge(data_train,new_features)
data_test_full = pd.merge(data_test,new_features)

In [38]:
data_train_full.index=data_train_full['client_id']
del data_train_full['client_id']

In [40]:
data_test_full.index=data_test_full['client_id']
del data_test_full['client_id']

In [45]:
X = data_train_full.drop(['target'],axis='columns')

In [46]:
X.columns

Index(['first_issue_date', 'first_redeem_date', 'age', 'gender',
       'treatment_flg', 'fake_age', 'transaction_count', 'av_sum',
       'last_transaction', 'store_count', 'alc_count', 'own_count'],
      dtype='object')

In [64]:
Cat_cols = ['gender','fake_age']
Num_cols = ['first_issue_date', 'first_redeem_date', 'age','transaction_count', 'av_sum',
       'last_transaction', 'store_count', 'alc_count', 'own_count']

In [54]:
enc = OneHotEncoder(sparse=False)
scaler = StandardScaler()

In [55]:
X_cat_encoded = pd.DataFrame(enc.fit_transform(X[Cat_cols]))

In [116]:
X_num_scaled = pd.DataFrame(scaler.fit_transform(X[Num_cols]),columns=Num_cols)

In [66]:
Test_cat = pd.DataFrame(enc.fit_transform(data_test_full[Cat_cols]))

In [118]:
Test_num =  pd.DataFrame(scaler.fit_transform(data_test_full[Num_cols]),columns=Num_cols)

In [119]:
test = pd.concat([Test_cat,Test_num],axis=1)

In [120]:
train =pd.concat([X_cat_encoded,X_num_scaled,data_train['treatment_flg']],axis=1)

In [121]:
train

,0,1,2,3,4,first_issue_date,first_redeem_date,age,transaction_count,av_sum,last_transaction,store_count,alc_count,own_count,treatment_flg
0,0.0,0.0,1.0,1.0,0.0,-0.769766,-0.774467,-0.028614,-0.909133,0.547241,0.342095,-0.511549,-0.373913,-0.701854,0
1,1.0,0.0,0.0,1.0,0.0,-1.344655,-1.946878,0.516484,0.669933,-0.505615,0.837526,0.816155,-0.178855,-0.179371,1
2,0.0,0.0,1.0,1.0,0.0,0.963771,0.382284,0.738561,-0.119600,-0.451077,-0.773153,0.091953,-0.178855,-0.440613,1
3,1.0,0.0,0.0,1.0,0.0,-0.945794,0.303902,-0.270880,-0.288786,-0.228108,0.664267,-0.028748,-0.373913,-0.388364,1
4,0.0,0.0,1.0,1.0,0.0,-0.210787,-0.747337,0.536673,-0.119600,2.506071,0.496630,-0.028748,-0.373913,1.388076,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200034,1.0,0.0,0.0,1.0,0.0,-0.211349,-0.602550,-0.230502,-0.627157,-0.517848,0.456418,-0.270148,-0.373913,-0.754102,0
200035,0.0,0.0,1.0,1.0,0.0,-0.569826,-0.880572,0.455918,0.557142,-0.396568,1.042361,-0.632249,-0.373913,0.604352,0
200036,1.0,0.0,0.0,1.0,0.0,0.109478,-0.468437,0.536673,-0.175995,-0.648813,0.142705,-0.752950,-0.373913,-0.910847,0
200037,1.0,0.0,0.0,1.0,0.0,-0.058513,-0.807931,0.617428,-0.232391,-0.850972,0.516491,-0.632249,-0.373913,-0.858599,1


In [79]:
logit = LogisticRegression(C=1, random_state=17, solver='liblinear')
logit.fit(train,data_train['target'])

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=17, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [126]:
test1 =test.copy()
test1['treatment_flg']=1
test0 = test.copy()
test0['treatment_flg']=0

In [86]:
preds1 = [i[1] for i in logit.predict_proba(test1)]
preds0 = [i[1] for i in logit.predict_proba(test0)]

In [135]:
uplift=[(preds1[i]-preds0[i]) for i in range(len(preds1))]

In [136]:
sol1 = pd.DataFrame({'client_id':data_test.client_id.values,'uplift':uplift})

In [137]:
sol1.index = sol1['client_id']
del sol1['client_id']

In [138]:
sol1

,uplift
client_id,
000048b7a6,0.056886
000073194a,0.037388
00007c7133,0.038690
00007f9014,0.017279
0000a90cf7,0.015631
...,...
fffeacbe3e,0.033306
fffece623e,0.035032
ffff3dfff8,0.052328


In [139]:
sol1.to_csv('sol2.csv')

In [113]:
from catboost import CatBoostClassifier

In [133]:
cbc = CatBoostClassifier(thread_count=2, random_state=17, silent=True)
cbc.fit(train,data_train['target'])

In [134]:
preds1 = [i[1] for i in cbc.predict_proba(test1)]
preds0 = [i[1] for i in cbc.predict_proba(test0)]